In [1]:
import requests 
import pandas as pd 
import json 
import numpy as np
import pprint

from bs4 import BeautifulSoup

In [2]:
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}

In [3]:
api_key = 'ac80e2c89884220d09d71b0042b91aebc5f4cd1f'

In [4]:
url = 'https://www.gamespot.com/api/reviews/'

In [5]:
url_params = {
    'api_key': api_key,
    'format': 'json',
    'limit': 100,
            }


# dict_keys(['publish_date', 'update_date', 'review_type', 'id', 'authors', 'title', 'image', 'score', 'deck', 'good', 'bad', 'body', 'lede', 'game', 'releases', 'site_detail_url'])

In [7]:
def data_call(url_params):
    response = requests.get(url,headers = header, params = url_params)
    res_json = json.loads(response.text)
    
    return res_json['results']



text = data_call(url_params)
print(text[0])


{'publish_date': '1996-05-01 12:00:00', 'update_date': '1996-05-01 12:00:00', 'review_type': 'primary', 'id': 2531694, 'authors': 'Jeff Sengstack', 'title': 'Under a Killing Moon Review', 'image': {'square_tiny': 'https://gamespot1.cbsistatic.com/uploads/square_tiny/469/4693985/2305502-default-art--kubrick.jpg', 'screen_tiny': 'https://gamespot1.cbsistatic.com/uploads/screen_tiny/469/4693985/2305502-default-art--kubrick.jpg', 'square_small': 'https://gamespot1.cbsistatic.com/uploads/square_small/469/4693985/2305502-default-art--kubrick.jpg', 'original': 'https://gamespot1.cbsistatic.com/uploads/original/469/4693985/2305502-default-art--kubrick.jpg'}, 'score': '6.9', 'deck': 'Under a Killing Moon is an excellently wrapped package with some less than compelling contents.', 'good': '', 'bad': '', 'body': '<p>Under a Killing Moon (UAKM) is an excellently wrapped package with some less than compelling contents.</p>  <p>You play Tex Murphy, the bumbling, disheveled detective from two previou

In [8]:
columns = ['Title', 'Platform', 'Rating']
tester = pd.DataFrame(columns=columns)
tester.to_csv('gamespotdata.csv')

In [9]:
print(len(text))

100


In [10]:
results = []
def parse_data(lst):
    for data in lst:
        if 'releases' in data.keys():
            for release in data['releases']:
                if 'Xbox One' in release['platform']:
                    obj = {'title':data['title'], 'platforms': release['platform'], 'score': data['score']}
                    results.append(obj)
    
    return results       




In [11]:
results

[]

In [13]:
cur = 0

#set up a while loop to go through and grab the result 
while cur < 15049:
    url_params['offset'] = cur
    info = data_call(url_params)
    parse_data(info)
    
    cur += 100

In [61]:
df = pd.DataFrame(results)

In [67]:
titles = []
for title in df['title']:
    titles.append(title.replace(' Review',''))
        

In [69]:
df['Title'] = titles

In [73]:
df.drop(columns='title', inplace=True)

In [12]:
df

NameError: name 'df' is not defined